In [1]:
import os
import igraph as ig
import plotly.plotly as py
from plotly.graph_objs import *
import sys
sys.path.append(os.path.abspath(".."))
from urlembed.util.seqmanager import *
from urlembed.util.plotter import *
from __future__ import print_function

<div style="text-align:center"><h1> NO-COSTRAINT GRAPH </h1></div>

In [2]:
"""g = ig.Graph(directed=True)
g.add_vertices(10)
g.add_edge(1, 2)
g.add_edge(1, 3)
g.add_edge(4, 5)
g.delete_vertices([0, 8, 9])

print([i.index for i in g.vs])
for i in g.es:
    print i.source, i.target"""

'g = ig.Graph(directed=True)\ng.add_vertices(10)\ng.add_edge(1, 2)\ng.add_edge(1, 3)\ng.add_edge(4, 5)\ng.delete_vertices([0, 8, 9])\n\nprint([i.index for i in g.vs])\nfor i in g.es:\n    print i.source, i.target'

In [3]:
nocostraint_path        = os.getcwd() + "/../dataset/cs.illinois.edu_NoConstraint.words1000.depth10/"
nocostraint_urlmap_path = nocostraint_path + "urlsMap.txt"
nocostraint_edges_path  = nocostraint_path + "edges.txt"

codeurlmap_nocostraint      = get_urlmap(nocostraint_urlmap_path)
nocostraint_graph       = graph_from_file(nocostraint_edges_path, codeurlmap_nocostraint)

print("Number of vertices:", len(nocostraint_graph.vs))
print("Number of edges:",    len(nocostraint_graph.es))

Number of vertices: 807
Number of edges: 16993


In [4]:
listcostraint_path        = os.getcwd() + "/../dataset/cs.illinois.edu_ListConstraint.words1000.depth10/"
listcostraint_urlmap_path = listcostraint_path + "urlsMap.txt"
listcostraint_edges_path  = listcostraint_path + "edges.txt"

urlmap_listcostraint      = get_urlmap(listcostraint_urlmap_path)
listcostraint_graph       = graph_from_file(listcostraint_edges_path, urlmap_listcostraint)

print("Number of vertices:", len(listcostraint_graph.vs))
print("Number of edges:",    len(listcostraint_graph.es))

Number of vertices: 1090
Number of edges: 19742


In [13]:
gt = GroundTruth(os.getcwd() + "/../dataset/ground_truth/urlToMembership.txt")
ground_truth = [int(gt.get_groundtruth(codeurlmap_nocostraint[key])) for key in codeurlmap_nocostraint]

In [30]:
nocostraint_graph.delete_vertices([vertex.index for vertex in nocostraint_graph.vs if vertex["name"] == "missing"])

for vertex in nocostraint_graph.vs:
    vertex["color"] = get_color(int(gt.get_groundtruth(vertex["name"])))

In [31]:
nocostraint_fig = graph3d_plot(nocostraint_graph, "No-costraint network")
py.iplot(nocostraint_fig, filename="No-costraint network")

The draw time for this plot will be slow for clients without much RAM.


<div>
    <a href="https://plot.ly/~chrispolo/44/" target="_blank" title="No-costraint network" style="display: block; text-align: center;"><img src="https://plot.ly/~chrispolo/44.png" alt="No-costraint network" style="max-width: 100%;width: 1000px;"  width="1000" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="chrispolo:44"  src="https://plot.ly/embed.js" async></script>
</div>


<div style="text-align:center"><h1> RANDOM-WALK-FROM-HOMEPAGE GRAPH </h1></div>

In [7]:
rwfh_path = os.getcwd() + "/dataset/depth-100k/seqLen-10/cs.illinois.eduRandomWalkFromHomepage.depth.100000.seqLen.10/"
rwfh_map_path = rwfh_path + "sequencesMapUrl.txt"
rwfh_seq_path = rwfh_path + "sequencesIDs.txt"


rwfh_sequence_map = get_sequence_map(rwfh_map_path)
rwfh_sequences = get_sequences(rwfh_seq_path, 1)

rwfh_graph = create_graph(rwfh_sequences, rwfh_sequence_map)
print len(rwfh_graph.vs), len(rwfh_graph.es)

SyntaxError: invalid syntax (<ipython-input-7-5f0a94552c4b>, line 10)

In [ ]:
rwfh_fig = graph_plot(rwfh_graph, "Random walk from Homepage network")
py.iplot(rwfh_fig, filename="Random walk from Homepage network")

<div style="text-align:center"><h1> RANDOM-WALK-WITH-LISTS GRAPH </h1></div>

In [ ]:
rwwl_path = os.getcwd() + "/dataset/depth-100k/seqLen-10/cs.illinois.eduRandomWalkLists.depth.100000.seqLen.10/"
rwwl_map_path = rwwl_path + "sequencesMapUrl.txt"
rwwl_seq_path = rwwl_path + "sequencesIDs.txt"


rwwl_sequence_map = get_sequence_map(rwwl_map_path)
rwwl_sequences = get_sequences(rwwl_seq_path, 1)

rwwl_graph = create_graph(rwwl_sequences, rwwl_sequence_map)
print len(rwwl_graph.vs), len(rwwl_graph.es)

In [ ]:
rwwl_fig = graph_plot(rwwl_graph, "Random walk with Lists network")
py.iplot(rwwl_fig, filename="Random walk with Lists network")

<div style="text-align:center"><h1> MANUALLY-COLORED-RWWL GRAPH </h1></div>

In [ ]:
color_rwwl_path = rwwl_path + "sequencesMapUrl-manually-clusterized.txt"
rwwl_sequences_wc = get_sequences(rwwl_seq_path, 1)
rwwl_tuplist = get_sequence_tuple_list(color_rwwl_path)

rwwl_graph_wc = create_graph(rwwl_sequences_wc, rwwl_tuplist)
print len(rwwl_graph_wc.vs), len(rwwl_graph_wc.es)

In [ ]:
rwmc = graph_plot(rwwl_graph_wc, "RWWL network - Manually colored")
py.iplot(rwmc, filename="RWWL network - Manually colored")

<div style="text-align:center"><h1> CLUSTERING </h1></div>

In [ ]:
vertex_dendogram = rwwl_graph.community_fastgreedy()
vertex_clustering = vertex_dendogram.as_clustering(8)

type(vertex_clustering.membership)

In [ ]:
print vertex_clustering.sizes()
print "modularity: ", vertex_clustering.modularity

In [ ]:
print type(vertex_clustering.membership[0])

In [ ]:
for i in range(len(rwwl_graph.vs)):
    rwwl_graph.vs[i]["color"] = get_color(vertex_clustering.membership[i])
    # print rw_graph.vs[i]["name"], vertex_clustering.membership[i]

In [ ]:
rwwl_clust_fig = graph_plot(rwwl_graph, "Random walk with lists - Modularity clustering")
py.iplot(rwwl_clust_fig, filename="Random walk with lists - Modularity clustering")

<div style="text-align:center"><h1> SINGLE CLUSTER </h1></div>

In [ ]:
second_clust_subgraph = vertex_clustering.subgraph(1)
second_clust_fig = graph_plot(second_clust_subgraph, "Single cluster")

py.iplot(second_clust_fig, filename="Single cluster")

In [ ]:
# for v in second_clust_subgraph.vs:
    # print v["name"]